In [2]:
# Importación de bibliotecas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

# Cargar datos históricos y nuevos
tabla_nuevos = pd.read_csv('https://raw.githubusercontent.com/juancamiloespana/LEA3_FIN/main/data/datos_nuevos_creditos.csv')
tabla_hist = pd.read_csv('https://raw.githubusercontent.com/juancamiloespana/LEA3_FIN/main/data/datos_historicos.csv')

# Información inicial de los datos
print("Datos históricos:")
print(tabla_hist.info())
print("\nDatos nuevos:")
print(tabla_nuevos.info())


Datos históricos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ID                               10000 non-null  int64  
 1   CreditScore                      10000 non-null  int64  
 2   DebtRatio                        10000 non-null  float64
 3   Assets                           10000 non-null  int64  
 4   Age                              10000 non-null  int64  
 5   NumberOfDependents               10000 non-null  int64  
 6   NumberOfOpenCreditLinesAndLoans  10000 non-null  int64  
 7   MonthlyIncome                    10000 non-null  int64  
 8   NumberOfTimesPastDue             10000 non-null  int64  
 9   EmploymentLength                 10000 non-null  int64  
 10  HomeOwnership                    10000 non-null  object 
 11  Education                        10000 non-null  object 
 12  M

In [3]:
# Eliminamos columnas no útiles
tabla_hist.drop(columns=['ID', 'HomeOwnership', 'Education', 'MaritalStatus'], inplace=True)

# Separación de variables explicativas (X) y objetivo (y)
xtrain = tabla_hist.iloc[:, :-1]  # Todas las columnas excepto la última
ytrain = tabla_hist["NoPaidPerc"]  # Columna objetivo

# Identificar columnas numéricas y categóricas
colum_numericas = xtrain.select_dtypes(exclude='object').columns
colum_categoricas = xtrain.select_dtypes(include='object').columns

# Verificar dimensiones después de la separación
print("Dimensiones de X (variables explicativas):", xtrain.shape)
print("Dimensiones de y (variable objetivo):", ytrain.shape)
print("Columnas numéricas:", colum_numericas)
print("Columnas categóricas:", colum_categoricas)


Dimensiones de X (variables explicativas): (10000, 10)
Dimensiones de y (variable objetivo): (10000,)
Columnas numéricas: Index(['CreditScore', 'DebtRatio', 'Assets', 'Age', 'NumberOfDependents',
       'NumberOfOpenCreditLinesAndLoans', 'MonthlyIncome',
       'NumberOfTimesPastDue', 'EmploymentLength', 'YearsAtCurrentAddress'],
      dtype='object')
Columnas categóricas: Index([], dtype='object')


In [4]:


# Escalado de variables numéricas
scaler = StandardScaler()
xtrain_scaled = scaler.fit_transform(xtrain)

# Confirmamos las dimensiones después del escalado
print("Dimensiones de X escalado:", xtrain_scaled.shape)


Dimensiones de X escalado: (10000, 10)


In [5]:

# Modelo inicial
rfr = RandomForestRegressor(random_state=42)

# Validación cruzada
scores = cross_val_score(rfr, xtrain_scaled, ytrain, cv=5, scoring='r2')

# Mostrar resultados
print("Puntajes de validación cruzada (R2):", scores)
print("Promedio R2:", np.mean(scores))


Puntajes de validación cruzada (R2): [0.21303382 0.20633444 0.260819   0.23697766 0.14162505]
Promedio R2: 0.21175799435309167


In [6]:


# Modelos para la selección de variables
modelos = [LinearRegression(), DecisionTreeRegressor(), RandomForestRegressor(random_state=42), GradientBoostingRegressor(random_state=42)]

# Definimos una función auxiliar para seleccionar las variables importantes )
def sel_variables(modelos, x, y, threshold):
    importancia = np.zeros(x.shape[1])
    for modelo in modelos:
        modelo.fit(x, y)
        if hasattr(modelo, 'feature_importances_'):
            importancia += modelo.feature_importances_
        elif hasattr(modelo, 'coef_'):
            importancia += np.abs(modelo.coef_)
    importancia = importancia / len(modelos)  # Promedio de importancia
    umbral = eval(threshold.replace("mean", str(np.mean(importancia))))
    indices_seleccionados = np.where(importancia >= umbral)[0]
    return x.columns[indices_seleccionados]

# Selección de variables importantes
xtrain_df = pd.DataFrame(xtrain_scaled, columns=colum_numericas)  # Convertimos a DataFrame
var_names = sel_variables(modelos, xtrain_df, ytrain, threshold="2.2*mean")

# Reducir el conjunto de entrenamiento a las variables seleccionadas
xtrain_reduced = xtrain_df[var_names]

# Mostrar las variables seleccionadas
print("Variables seleccionadas:", var_names)
print("Dimensiones de X reducido:", xtrain_reduced.shape)


Variables seleccionadas: Index(['DebtRatio'], dtype='object')
Dimensiones de X reducido: (10000, 1)


In [7]:
# Validación cruzada con las variables seleccionadas
scores_reduced = cross_val_score(rfr, xtrain_reduced, ytrain, cv=5, scoring='r2')

# Mostrar resultados
print("Puntajes de validación cruzada (R2) con variables seleccionadas:", scores_reduced)
print("Promedio R2 con variables seleccionadas:", np.mean(scores_reduced))


Puntajes de validación cruzada (R2) con variables seleccionadas: [-0.37156652 -0.43194695 -0.38365393 -0.3367181  -0.34037359]
Promedio R2 con variables seleccionadas: -0.37285181766873726


In [8]:
# Ajustar el umbral para incluir más variables
var_names_adjusted = sel_variables(modelos, xtrain_df, ytrain, threshold="1.0*mean")  # Reducimos el umbral
xtrain_reduced_adjusted = xtrain_df[var_names_adjusted]

# Validación cruzada con el nuevo conjunto reducido
scores_adjusted = cross_val_score(rfr, xtrain_reduced_adjusted, ytrain, cv=5, scoring='r2')

# Mostrar resultados
print("Nuevas variables seleccionadas:", var_names_adjusted)
print("Dimensiones de X reducido ajustado:", xtrain_reduced_adjusted.shape)
print("Puntajes de validación cruzada (R2) con variables ajustadas:", scores_adjusted)
print("Promedio R2 con variables ajustadas:", np.mean(scores_adjusted))


Nuevas variables seleccionadas: Index(['DebtRatio', 'Age', 'EmploymentLength'], dtype='object')
Dimensiones de X reducido ajustado: (10000, 3)
Puntajes de validación cruzada (R2) con variables ajustadas: [ 0.01394739  0.026485    0.04559744  0.02211818 -0.02914297]
Promedio R2 con variables ajustadas: 0.015801006338135415


In [9]:
# Validación cruzada con todas las variables
scores_all = cross_val_score(rfr, xtrain_scaled, ytrain, cv=5, scoring='r2')

# Mostrar resultados
print("Puntajes de validación cruzada (R2) con todas las variables:", scores_all)
print("Promedio R2 con todas las variables:", np.mean(scores_all))


Puntajes de validación cruzada (R2) con todas las variables: [0.21303382 0.20633444 0.260819   0.23697766 0.14162505]
Promedio R2 con todas las variables: 0.21175799435309167


In [10]:


# Espacio de búsqueda reducido
param_grid = {
    'n_estimators': [50, 100],       # Reducimos el número de árboles
    'max_depth': [5, 10],            # Reducimos las profundidades
    'min_samples_split': [2, 5],     # Menos opciones para división mínima
    'min_samples_leaf': [1, 2]       # Menos opciones para hojas mínimas
}

# Configuramos GridSearchCV
grid_search = GridSearchCV(RandomForestRegressor(random_state=42),
                           param_grid=param_grid,
                           scoring='r2',
                           cv=3,          # Reducimos el número de folds
                           n_jobs=-1,     # Paralelización
                           verbose=3)     # Mostrar progreso

# Ajustamos el modelo
grid_search.fit(xtrain_scaled, ytrain)

# Mejor modelo y parámetros
best_model = grid_search.best_estimator_
print("Mejores parámetros encontrados:", grid_search.best_params_)


Fitting 3 folds for each of 16 candidates, totalling 48 fits


[CV 1/3] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.182 total time=   0.8s
[CV 2/3] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.218 total time=   0.8s
[CV 3/3] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.136 total time=   0.8s
[CV 2/3] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=50;, score=0.218 total time=   0.8s
[CV 1/3] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=50;, score=0.182 total time=   0.7s
[CV 3/3] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=50;, score=0.136 total time=   0.8s
[CV 2/3] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.221 total time=   1.5s
[CV 1/3] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.183 total time=   1.6s
[CV 1/3] END max_depth=5, min_samples_leaf=2, min_samples_split=2, n_e

In [11]:
# Construimos el modelo final con los mejores parámetros
best_rfr = RandomForestRegressor(max_depth=10, 
                                 min_samples_leaf=2, 
                                 min_samples_split=5, 
                                 n_estimators=100, 
                                 random_state=42)

# Validación cruzada con el modelo optimizado
scores_best = cross_val_score(best_rfr, xtrain_scaled, ytrain, cv=5, scoring='r2')

# Mostramos los resultados
print("Puntajes de validación cruzada (R2) con el modelo optimizado:", scores_best)
print("Promedio R2 con el modelo optimizado:", np.mean(scores_best))


Puntajes de validación cruzada (R2) con el modelo optimizado: [0.22001679 0.2306006  0.26748879 0.24495169 0.14458561]
Promedio R2 con el modelo optimizado: 0.22152869561174


In [16]:
# Alinear las columnas de los datos nuevos con las de los datos históricos
x_nuevos = tabla_nuevos.drop(columns=['ID'])  # Eliminamos la columna ID

# Codificar las columnas categóricas usando los valores únicos de los datos históricos
categorical_cols = ['HomeOwnership', 'Education', 'MaritalStatus']
x_nuevos = pd.get_dummies(x_nuevos, columns=categorical_cols, drop_first=True)

# Alinear las columnas de los datos nuevos con las del modelo (rellenamos las faltantes con ceros)
x_nuevos = x_nuevos.reindex(columns=xtrain_df.columns, fill_value=0)

# Escalar los datos nuevos con el mismo escalador
x_nuevos_scaled = scaler.transform(x_nuevos)

# Generar predicciones con el modelo optimizado
best_rfr.fit(xtrain_scaled, ytrain)  # Entrenamos el modelo con todos los datos históricos
predicciones = best_rfr.predict(x_nuevos_scaled)

# Sumar 15 al interés base (según requerimiento)
predicciones_ajustadas = predicciones 

# Crear un DataFrame con los resultados y columnas personalizadas
resultados = pd.DataFrame({
    "ID": tabla_nuevos["ID"],
    "int_rc": predicciones_ajustadas
})

# Guardar los resultados en un archivo CSV delimitado por comas
resultados.to_csv("nuevogrupo_7.csv", index=False)
print("Predicciones guardadas en 'grupo_7.csv'.")


Predicciones guardadas en 'grupo_7.csv'.
